# labelme2coco

In [1]:
import json
from glob import glob
import mmcv

In [4]:
ann_path = './data/train/*.txt'
ann_files = glob(ann_path)

In [3]:
def labelmetxt2coco(anns):
    cars = {}
    # 숫자로 해도 되지만, 클래스 확인하기 위해 차종으로 변경
    classes = ["chevrolet_malibu_sedan_2012_2016", "chevrolet_malibu_sedan_2017_2019", "chevrolet_spark_hatchback_2016_2021", "chevrolet_trailblazer_suv_2021_", "chevrolet_trax_suv_2017_2019", "genesis_g80_sedan_2016_2020", "genesis_g80_sedan_2021_", "genesis_gv80_suv_2020_", "hyundai_avante_sedan_2011_2015", "hyundai_avante_sedan_2020_", "hyundai_grandeur_sedan_2011_2016", "hyundai_grandstarex_van_2018_2020", "hyundai_ioniq_hatchback_2016_2019", "hyundai_sonata_sedan_2004_2009", "hyundai_sonata_sedan_2010_2014", "hyundai_sonata_sedan_2019_2020", "kia_carnival_van_2015_2020", "kia_carnival_van_2021_", "kia_k5_sedan_2010_2015", "kia_k5_sedan_2020_", "kia_k7_sedan_2016_2020", "kia_mohave_suv_2020_", "kia_morning_hatchback_2004_2010", "kia_morning_hatchback_2011_2016", "kia_ray_hatchback_2012_2017", "kia_sorrento_suv_2015_2019", "kia_sorrento_suv_2020_", "kia_soul_suv_2014_2018", "kia_sportage_suv_2016_2020", "kia_stonic_suv_2017_2019", "renault_sm3_sedan_2015_2018", "renault_xm3_suv_2020_", "ssangyong_korando_suv_2019_2020", "ssangyong_tivoli_suv_2016_2020"]
    cars["categories"] = [{"id":i,"name":cat,"supercategory":"none"} for i, cat in enumerate(classes)]
    cars["images"] = []
    cars["annotations"] = []
    cnt_ann = 0
    for i, ann in enumerate(anns):
        img_path = ann.replace("/data", "").replace("txt", "png")
        #h, w, _ = cv2.imread(img_path).shape
        cars["images"].append({"id":i,"height":1040,"width":1920,"file_name":img_path})

        f_ann = open(ann, "r")
        for line in f_ann.readlines():
            data = line.split()
            cat = int(float(data[0]))
            pt1x = int(data[1])
            pt1y = int(data[2])
            #pt2x = int(data[3])    not to use
            #pt2y = int(data[4])
            pt3x = int(data[5])
            pt3y = int(data[6])
            #pt4x = int(data[7])
            #pt4y = int(data[8])
            x = pt1x
            y = pt1y
            width = pt3x - pt1x
            height = pt3y - pt1y
            area = width * height
            cars["annotations"].append({"id": cnt_ann,
                                        "image_id": i,
                                        "category_id": cat,
                                        "bbox": [x, y, width, height],
                                        "area": area,
                                        "segmentation": [],
                                        "iscrowd": 0})
            cnt_ann += 1
        
    return cars

In [4]:
with open('./data/annotations/train.json', "w") as f:
    json.dump(labelmetxt2coco(ann_files), f, ensure_ascii=False, indent=4)

# test ann_file 생성

In [2]:

test_img_path = './data/test/*'
test_img_files = glob(test_img_path)

In [3]:
img_infos = []
for idx, img_path in enumerate(test_img_files):
    filename = img_path.replace("/data", "")
    height, width = mmcv.imread(str(img_path)).shape[:2]
    img_infos.append(
        dict(id=idx, file_name=filename, width=width, height=height))

print(f'dataset size: {len(img_infos)}')

dataset size: 3400


In [4]:
test = {}
classes = ["chevrolet_malibu_sedan_2012_2016", "chevrolet_malibu_sedan_2017_2019", "chevrolet_spark_hatchback_2016_2021", "chevrolet_trailblazer_suv_2021_", "chevrolet_trax_suv_2017_2019", "genesis_g80_sedan_2016_2020", "genesis_g80_sedan_2021_", "genesis_gv80_suv_2020_", "hyundai_avante_sedan_2011_2015", "hyundai_avante_sedan_2020_", "hyundai_grandeur_sedan_2011_2016", "hyundai_grandstarex_van_2018_2020", "hyundai_ioniq_hatchback_2016_2019", "hyundai_sonata_sedan_2004_2009", "hyundai_sonata_sedan_2010_2014", "hyundai_sonata_sedan_2019_2020", "kia_carnival_van_2015_2020", "kia_carnival_van_2021_", "kia_k5_sedan_2010_2015", "kia_k5_sedan_2020_", "kia_k7_sedan_2016_2020", "kia_mohave_suv_2020_", "kia_morning_hatchback_2004_2010", "kia_morning_hatchback_2011_2016", "kia_ray_hatchback_2012_2017", "kia_sorrento_suv_2015_2019", "kia_sorrento_suv_2020_", "kia_soul_suv_2014_2018", "kia_sportage_suv_2016_2020", "kia_stonic_suv_2017_2019", "renault_sm3_sedan_2015_2018", "renault_xm3_suv_2020_", "ssangyong_korando_suv_2019_2020", "ssangyong_tivoli_suv_2016_2020"]
test["categories"] = [{"id":i,"name":cat,"supercategory":"none"} for i, cat in enumerate(classes)]
test["images"] = img_infos

with open('./data/annotations/test.json', "w") as f:
    json.dump(test, f, ensure_ascii=False, indent=4)

# Split data

In [5]:
import numpy as np
from sklearn.model_selection import StratifiedGroupKFold

In [6]:
# annotation = {train.json dataset file 경로}
annotation = './data/annotations/train.json'

with open(annotation) as f: data = json.load(f)

var = [(ann['image_id'], ann['category_id']) for ann in data['annotations']]
X = np.ones((len(data['annotations']),1))
y = np.array([v[1] for v in var])
groups = np.array([v[0] for v in var])

cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=777)

for train_idx, val_idx in cv.split(X, y, groups):
    print("TRAIN:", groups[train_idx]) # image_id
    print(" ", y[train_idx])           # category_id
    print(" TEST:", groups[val_idx])
    print(" ", y[val_idx])

TRAIN: [   0    0    1 ... 6480 6480 6480]
  [ 5 17 32 ... 25 17 21]
 TEST: [   2    2    2 ... 6479 6479 6479]
  [23 22 24 ... 24 32 32]
TRAIN: [   0    0    2 ... 6479 6479 6479]
  [ 5 17 23 ... 24 32 32]
 TEST: [   1    1    1 ... 6480 6480 6480]
  [32 26 24 ... 25 17 21]
TRAIN: [   0    0    1 ... 6480 6480 6480]
  [ 5 17 32 ... 25 17 21]
 TEST: [  10   10   10 ... 6473 6473 6473]
  [16  3 23 ...  8 15 18]
TRAIN: [   0    0    1 ... 6480 6480 6480]
  [ 5 17 32 ... 25 17 21]
 TEST: [   7    7    7 ... 6478 6478 6478]
  [ 3  1  6 ... 31 10 31]
TRAIN: [   1    1    1 ... 6480 6480 6480]
  [32 26 24 ... 25 17 21]
 TEST: [   0    0    4 ... 6475 6475 6475]
  [ 5 17  5 ... 15 21 23]


In [7]:
from collections import Counter
import pandas as pd

In [8]:
def get_distribution(y):
    y_distr = Counter(y)
    y_vals_sum = sum(y_distr.values())

    return [f'{y_distr[i]/y_vals_sum:.2%}' for i in range(np.max(y) +1)]

In [9]:
distrs = [get_distribution(y)]
index = ['training set']

In [10]:
for fold_ind, (train_idx, val_idx) in enumerate(cv.split(X, y, groups)):
    train_y, val_y = y[train_idx], y[val_idx]
    train_gr, val_gr = groups[train_idx], groups[val_idx]

    assert len(set(train_gr) & set(val_gr)) == 0
    
    distrs.append(get_distribution(train_y))
    distrs.append(get_distribution(val_y))
    index.append(f'train - fold{fold_ind+1}')
    index.append(f'val - fold{fold_ind+1}')

In [11]:
categories = [d['name'] for d in data['categories']]
pd.DataFrame(distrs, index=index, columns = [categories[i] for i in range(np.max(y) + 1)])

,chevrolet_malibu_sedan_2012_2016,chevrolet_malibu_sedan_2017_2019,chevrolet_spark_hatchback_2016_2021,chevrolet_trailblazer_suv_2021_,chevrolet_trax_suv_2017_2019,genesis_g80_sedan_2016_2020,genesis_g80_sedan_2021_,genesis_gv80_suv_2020_,hyundai_avante_sedan_2011_2015,hyundai_avante_sedan_2020_,...,kia_ray_hatchback_2012_2017,kia_sorrento_suv_2015_2019,kia_sorrento_suv_2020_,kia_soul_suv_2014_2018,kia_sportage_suv_2016_2020,kia_stonic_suv_2017_2019,renault_sm3_sedan_2015_2018,renault_xm3_suv_2020_,ssangyong_korando_suv_2019_2020,ssangyong_tivoli_suv_2016_2020
training set,2.94%,2.94%,2.94%,2.94%,2.94%,2.94%,2.94%,2.94%,2.94%,2.94%,...,2.94%,2.94%,2.94%,2.94%,2.94%,2.94%,2.94%,2.94%,2.94%,2.94%
train - fold1,3.01%,3.04%,3.04%,2.84%,2.92%,2.94%,3.03%,2.94%,2.94%,2.87%,...,2.80%,2.84%,2.78%,2.92%,2.98%,2.88%,2.96%,2.82%,2.96%,2.94%
val - fold1,2.67%,2.56%,2.56%,3.35%,3.02%,2.94%,2.58%,2.94%,2.94%,3.23%,...,3.49%,3.35%,3.58%,3.02%,2.79%,3.17%,2.85%,3.44%,2.88%,2.94%
train - fold2,2.98%,2.76%,3.00%,2.96%,2.98%,2.91%,2.87%,3.03%,2.92%,2.92%,...,2.94%,3.02%,2.93%,2.93%,3.03%,3.06%,3.00%,2.92%,2.88%,2.94%
val - fold2,2.80%,3.69%,2.71%,2.86%,2.77%,3.07%,3.24%,2.59%,3.04%,3.01%,...,2.95%,2.62%,2.98%,2.98%,2.59%,2.45%,2.71%,3.04%,3.18%,2.95%
train - fold3,2.94%,2.86%,2.90%,3.02%,3.02%,2.95%,2.90%,2.85%,3.04%,2.90%,...,3.05%,2.91%,3.06%,2.96%,2.94%,2.95%,2.93%,3.00%,2.92%,2.99%
val - fold3,2.94%,3.26%,3.09%,2.62%,2.62%,2.91%,3.12%,3.29%,2.53%,3.12%,...,2.50%,3.06%,2.47%,2.88%,2.94%,2.91%,3.00%,2.71%,3.03%,2.74%
train - fold4,2.84%,3.02%,2.81%,2.99%,2.91%,2.97%,2.99%,2.89%,2.90%,2.98%,...,2.97%,3.05%,3.01%,2.86%,2.86%,2.92%,2.91%,3.00%,3.02%,2.92%
val - fold4,3.34%,2.61%,3.46%,2.75%,3.08%,2.81%,2.75%,3.14%,3.11%,2.78%,...,2.84%,2.49%,2.67%,3.25%,3.28%,3.02%,3.08%,2.72%,2.64%,3.02%
train - fold5,2.94%,3.03%,2.95%,2.89%,2.87%,2.93%,2.92%,2.99%,2.90%,3.03%,...,2.95%,2.88%,2.92%,3.03%,2.90%,2.89%,2.91%,2.98%,2.93%,2.91%


In [12]:
# annotation = {dataset 경로/K-fold}
output_filename = "./data/K-fold"

In [13]:
for idx, (train_idx, val_idx) in enumerate(cv.split(X, y, groups)):
    train_images, val_images = [], []
    train_annotations, val_annotations = [], []
    for i in groups[train_idx]: # image_id
        train_images.append(data["images"][i].copy())
    for i in groups[val_idx]:   # image_id
        val_images.append(data["images"][i].copy())
    for annotation in data["annotations"]:
        if annotation["image_id"] in groups[val_idx]:
            val_annotations.append(annotation.copy())
        else:
            train_annotations.append(annotation.copy())

    train_split = {
            "images": train_images,
            "annotations": train_annotations,
            "info": data.get("info", {}),
            "licenses": data.get("licenses", []),
            "categories": data["categories"],
        }

    val_split = {
            "images": val_images,
            "annotations": val_annotations,
            "info": data.get("info", {}),
            "licenses": data.get("licenses", []),
            "categories": data["categories"],
        }
    
    output_files = []
    for split_type, split in zip(["train", "val"], [train_split, val_split]):
        output_files.append(output_filename + f"_{split_type}{idx+1}.json")
        with open(output_files[-1], "w") as f:
            json.dump(split, f, indent=2)